In [ ]:
import torch
import torch.nn.functional as F
print(torch.__version__)

'2.1.0+cu118'

In [ ]:
import time
import torch
import torch.nn.functional as F

# Set up the basic parameters for the model
batch_size = 32
sequence_length = 2048
dimensions = 64
number_of_heads = 8

# Creating query (q), key (k), and value (v) tensors
# These tensors are initialized with random values and moved to the GPU for faster processing
q = torch.randn(batch_size, number_of_heads, sequence_length, dimensions, dtype=torch.float16).cuda()
k = torch.randn(batch_size, number_of_heads, sequence_length, dimensions, dtype=torch.float16).cuda()
v = torch.randn(batch_size, number_of_heads, sequence_length, dimensions, dtype=torch.float16).cuda()

# Define the dropout rate and number of trials for benchmarking
dropout_rate = 0.2
num_trials = 10

# Standard Attention Computation
torch.cuda.synchronize()  # Synchronizes CPU and GPU to ensure accurate timing
start = time.time()  # Start timer
for i in range(num_trials):
    attn = q @ k.transpose(-2, -1)  # Compute attention scores
    attn = attn.softmax(dim=-1)  # Apply softmax to get probabilities
    attn = F.dropout(attn, p=dropout_rate, training=True)  # Apply dropout
    x = (attn @ v).transpose(1, 2)  # Apply attention to value and reshape
torch.cuda.synchronize()  # Ensure all GPU tasks are finished
end = time.time()  # End timer
print('Standard attention took {} seconds for {} trials'.format(end - start, num_trials))

# Flash Attention Computation
with torch.backends.cuda.sdp_kernel(
    enable_flash=True, enable_math=False, enable_mem_efficient=False
):
    torch.cuda.synchronize()  # Synchronizes CPU and GPU for accurate timing
    start = time.time()  # Start timer
    for i in range(num_trials):
        out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_rate)  # Compute attention using FlashAttention
    torch.cuda.synchronize()  # Ensure completion of all GPU tasks
    end = time.time()  # End timer
    print('Flash attention took {} seconds for {} trials'.format(end - start, num_trials))


Standard attention took 0.9052870273590088 seconds for 10 trials
Flash attention took 0.31307268142700195 seconds for 10 trials


In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.3 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.3.3-cp310-cp310-linux_x86_64.whl size=57075008 sha256=bcb63b64213ab61590b340b77de84e448a442e19c100480895194df39ad7673d
  Stored in directory: /root/.cache/pip/wheels/e5/e6/fa/941802ec61d1afd320d27160ab1db98e6dba65381f84b76d4a
Successfully built flash-attn


In [ ]:
from flash_attn import flash_attn_qkvpacked_func, flash_attn_func


In [ ]:
!git clone https://github.com/Dao-AILab/flash-attention.git

Cloning into 'flash-attention'...
remote: Enumerating objects: 4303, done.
remote: Counting objects: 100% (1785/1785), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 4303 (delta 1632), reused 1606 (delta 1590), pack-reused 2518
Receiving objects: 100% (4303/4303), 6.88 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (3007/3007), done.


In [ ]:
%cd flash-attention/

/content/flash-attention


In [ ]:
!pytest -q -s tests/test_flash_attn.py

Streaming output truncated to the last 5000 lines.
        q, k, v = [maybe_contiguous(x) for x in (q, k, v)]
>       out, q, k, v, out_padded, softmax_lse, S_dmask, rng_state = flash_attn_cuda.fwd(
            q,
            k,
            v,
            None,
            dropout_p,
            softmax_scale,
            causal,
            window_size[0],
            window_size[1],
            return_softmax,
            None,
        )
E       RuntimeError: FlashAttention only supports Ampere GPUs or newer.

/usr/local/lib/python3.10/dist-packages/flash_attn/flash_attn_interface.py:47: RuntimeError
_____________________ test_flash_attn_qkvpacked[0.0-200-111-True-True-dtype0] ______________________

seqlen = 200, d = 111, dropout_p = 0.0, causal = True, local = True, dtype = torch.float16

    @pytest.mark.parametrize("dtype", ([torch.float16] if is_sm75 else [torch.float16, torch.bfloat16]))
    # @pytest.mark.parametrize("dtype", [torch.float16])
    @pytest.mark.parametrize("loca